## Setup






In [50]:
import time
import shutil
import pickle
import os
import numpy as np
import google.colab as colab
import zipfile
import random
import json
from collections import defaultdict
import cv2
import nltk
nltk.download('punkt')
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Mount Google Drive

In [0]:
def mount_google_drive():
	'''
	# Functionality
		Mount google drive. Since colab does not save files, we want to make it easier to directly access files in google drive.
	# Arguments
		Nothing
	# Returns
		drive_root: the working directory mounted
	'''
	mount_directory = "/content/gdrive"
	drive = colab.drive
	drive.mount(mount_directory, force_remount=True)
	drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0]
	return drive_root

In [3]:
ROOT_DIR =  mount_google_drive()
CHECKPOINT_ROOT = ROOT_DIR+ "/FYDP/captioning/checkpoints/"
DATASET_DIR = ROOT_DIR + "/FYDP/Dataset/"


def get_checkpoint_path(epoch=None):
    if epoch is None:
        return os.path.abspath(CHECKPOINT_ROOT + "weights")
    else:
        return os.path.abspath(CHECKPOINT_ROOT + "weights_{}".format(epoch))
      
# example of checkpoint dir
print(get_checkpoint_path(4))

Mounted at /content/gdrive
/content/gdrive/My Drive/FYDP/captioning/checkpoints/weights_4


In [0]:
!rm -r utils

In [5]:
shutil.copytree(ROOT_DIR + "/FYDP/utils/", "utils/")
shutil.copy(ROOT_DIR + "/FYDP/models.py", "models.py")

'models.py'

In [6]:
from utils.file_utils import *
from utils.image_utils import *
from utils.generator_utils import *
from utils.tqdm_utils import *
from utils.keras_utils import *

Using TensorFlow backend.


## Infersent Init
Never run this section unless **First Run**. This will take tons of time and disk space

In [0]:
#https://github.com/facebookresearch/InferSent

In [0]:
# !mkdir GloVe
# !curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip GloVe/glove.840B.300d.zip -d GloVe/
# !mkdir fastText
# !curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# !unzip fastText/crawl-300d-2M.vec.zip -d fastText/

In [0]:
# !mkdir encoder
# !curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
# !curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.3M      0  0:00:14  0:00:14 --:--:-- 12.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.4M      0  0:00:14  0:00:14 --:--:-- 11.9M


In [0]:
# shutil.copytree("GloVe/", ROOT_DIR + "/FYDP/nli_utils/GloVe/")
# shutil.copytree("fastText/", ROOT_DIR + "/FYDP/nli_utils/fastText/")
# shutil.copytree("encoder/", ROOT_DIR + "/FYDP/nli_utils/encoder/")

## Infersent Fitting

Store trained Infersent model and numpy embeddings.


In [0]:
from models import InferSent

def build_nli_net():
  V = 2
  MODEL_PATH = ROOT_DIR + '/FYDP/nli_utils/encoder/infersent%s.pkl' % V
  params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                  'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
  infersent = InferSent(params_model)
  infersent.load_state_dict(torch.load(MODEL_PATH))
  return infersent

In [0]:
# 1) Load our pre-trained model (in encoder/):
infersent = build_nli_net()

In [0]:
# 2) Set word vector path for the model:
W2V_PATH = ROOT_DIR + '/FYDP/nli_utils/fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [13]:
# NOT necessary
# 3) Build the vocabulary of word vectors (i.e keep only those needed):
# infersent.build_vocab(sentences, tokenize=True)
infersent.build_vocab_k_words(K=100000)

Vocab size : 100000


In [0]:
# 4) Encode your sentences (list of n sentences):
for file_name in get_all_files_from_dir(DATASET_DIR + "songs/"):
  with open(file_name, 'r') as f:
    sentences = f.readlines()
  embeddings = infersent.encode(sentences, tokenize=True)
  np.save(DATASET_DIR + "song_embeds/" + file_name.split("/")[-1].split(".")[0] + ".npy", embeddings)

## Test

Load pretrained model and embeddings

In [0]:
def cosine(u, v):
  # compute the similarity between two embeddings
  # u and v are matrices!
    return np.einsum('ij,ij->i', u, v) / ((np.linalg.norm(u, axis=1) * np.linalg.norm(v, axis=1)))

In [24]:
cosine(infersent.encode(['the cat eats.']), infersent.encode(['the cat is hungry.']))

array([0.75352097], dtype=float32)

## Song Ranking Experiment


In [0]:
def compute_score(caption, song_emb_files):
  best_song_index = 0
  best_score = 0
  caption_emb = infersent.encode([caption])
  for index, file_name in enumerate(song_emb_files):
    song_emb = np.load(file_name)
    score = np.max(cosine(np.repeat(caption_emb, song_emb.shape[0], axis=0), song_emb))
    if score > best_score:
      best_score = score
      best_song_index = index
  return best_song_index, best_score

In [44]:
tic = time.time()
caption = "It is snowy outside"
best_song_index, best_score = compute_score(caption, get_all_files_from_dir(DATASET_DIR + "song_embeds/"))
toc = time.time()
print("It takes", str(toc - tic), "seconds to forward pass", len(get_all_files_from_dir(DATASET_DIR + "song_embeds/")), "songs.")

It takes 0.1624903678894043 seconds to forward pass 26 songs.


In [52]:
with open(get_all_files_from_dir(DATASET_DIR + "songs/")[best_song_index], "r") as f:
  pprint(f.readlines())

['Oh, the weather outside is frightful\n',
 'But the fire is so delightful\n',
 "And since we've no place to go\n",
 'Let it snow, let it snow, let it snow\n',
 "Man it doesn't show signs of stoppin'\n",
 "And I brought me some corn for poppin'\n",
 'The lights are turned way down low\n',
 'Let it snow, let it snow, let it snow\n',
 'When we finally kiss good-night\n',
 "How I'll hate going out in the storm\n",
 'But if you really hold me tight\n',
 "All the way home I'll be warm\n",
 'And the fire is slowly dying\n',
 "And, my dear, we're still good-bye-ing\n",
 'But as long as you love me so\n',
 'Let it snow, let it snow, and snow\n',
 'When we finally kiss good-night\n',
 "How I'll hate going out in the storm\n",
 'But if you really grab me tight\n',
 "All the way home I'll be warm\n",
 'Oh the fire is slowly dying\n',
 "And, my dear, we're still good-bye-ing\n",
 'But as long as you love me so\n',
 'Let it snow, let it snow, let it snow']
